## Exporting to CoreML
To use a Fireball model in an iOS application, we can use [exportToCoreMl](https://interdigitalinc.github.io/Fireball/html/source/model.html#fireball.model.Model.exportToCoreMl) method. This notebook shows how to use this function to create a CoreML model ready to be deployed in an iOS app. It assumes that a trained LeNet-5 model already exists in the ```Models``` directory. You can use the notebook [Handwritten Digit recognition (LeNet-5/MNIST)](LeNet5-MNIST.ipynb) to create and train a LeNet-5 model.

Fireball can also export models with reduced number of parameters, pruned models, and quatized models. Please refer to the following notebooks for more information:

- [Reducing number of parameters of LeNet-5 Model](LeNet5-MNIST-Reduce.ipynb)
- [Pruning LeNet-5 Model](LeNet5-MNIST-Prune.ipynb)
- [Quantizing LeNet-5 Model](LeNet5-MNIST-Quantize.ipynb)

Note: Fireball uses the [coremltools](https://github.com/apple/coremltools) python package to export CoreML models. 

## Load a pretrained model

In [1]:
from fireball import Model
from fireball.datasets.mnist import MnistDSet

testDs = MnistDSet.makeDatasets('test', batchSize=128)

# orgFileName = "Models/LeNet5.fbm"        # Original model
# orgFileName = "Models/LeNet5QR.fbm"      # Quantized - Retrained
# orgFileName = "Models/LeNet5PR.fbm"      # Pruned - Retrained
# orgFileName = "Models/LeNet5PRQR.fbm"    # Pruned - Retrained - Quantized - Retrained
# orgFileName = "Models/LeNet5RR.fbm"      # Reduced - Retrained
# orgFileName = "Models/LeNet5RRQR.fbm"    # Reduced - Retrained - Quantized - Retrained
# orgFileName = "Models/LeNet5RRPR.fbm"    # Reduced - Retrained - Pruned - Retrained
orgFileName = "Models/LeNet5RRPRQR.fbm"  # Reduced - Retrained - Pruned - Retrained - Quantized - Retrained

model = Model.makeFromFile(orgFileName, testDs=testDs, gpus='0')   
model.initSession()
model.printLayersInfo()

results = model.evaluate()


Reading from "Models/LeNet5RRPRQR.fbm" ... Done.
Creating the fireball model "LeNet-5" ... Done.
Metal device set to: Apple M1 Max

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 28x28x1      28 28 1       None                      0          
L1_CONV          28 28 1       KSP: 5 1 s               14 14 6       ReLU     MP(KSP):2 2 v    111        
L2_CONV          14 14 6       KSP: 5 1 v, LR8          5 5 16        ReLU     MP(KSP):2 2 v    844        
L3_FC            5 5 16        LR8                      120           ReLU                      2,441      
L4_FC            120           LR8                      84            ReLU                      987        
L5_FC            84                                     10            None                      539        
OUT


CoreML handles the pre-processing of the images inside the model. The arguments ```rgbBias``` and ```scale``` are used to tell CoreML how to do this pre-processing. The pre-processed image is calculated by CoreML as:
```
processedImage = image * scale + rgbBias
```

In [2]:
import os

cmlFile = orgFileName.replace('.fbm', '.mlmodel')

model.exportToCoreMl(cmlFile,
                     classNames=[str(i) for i in range(10)],  # List of labels (strings)
                     rgbBias=-.5,
                     scale=1.0/255)

orgFileSize = os.stat(orgFileName).st_size
print('\nOriginal Model File Size: {:,} bytes'.format(orgFileSize))
fileSize = os.stat(cmlFile).st_size
print('CoreML Model File Size: {:,} bytes ({:2.2%} of original)'.format(fileSize, fileSize/orgFileSize))


Exporting to CoreML model "Models/LeNet5RRPRQR.mlmodel" ... 
    Exported all 7 layers.                               
    Saving to "Models/LeNet5RRPRQR.mlmodel" ... Done.
Done (0.12 Sec.)

Original Model File Size: 9,071 bytes
CoreML Model File Size: 7,484 bytes (82.50% of original)


## Using netron to visualize the exported model
We can now visualize the model's network structure using the [netron](https://github.com/lutzroeder/netron) package.

In [3]:
import netron
import platform

if platform.system() == 'Darwin':      # Running on MAC
    netron.start(cmlFile)   
else:
    import socket
    hostIp = socket.gethostbyname(socket.gethostname())
    netron.start(cmlFile, address=(hostIp,8084))

Serving 'Models/LeNet5RRPRQR.mlmodel' at http://localhost:8081


## Evaluate the CoreML model (Runs only on Mac)
We can now evaluate the CoreML model. This also shows how to use a CoreML model for inference. Currently the CoreML runtime is only available on Mac. You also need the [pillow package](https://pypi.org/project/Pillow/) because CoreML only accepts images in this format. 

We could use fireball's implementation of MNIST dataset. But here we want to run and test the CoreML model without any dependency to Fireball libraty.

In [4]:
assert platform.system() == 'Darwin'

# Load MNIST Test Dataset:
import numpy as np
import struct

def getDataset(imagesFileName, labelsFileName):
    file = open(imagesFileName, mode='rb')
    header = file.read(16)
    magic, numImages, imageWidth, imageHeight = struct.unpack(">iiii", header)
    assert magic == 2051, "Error: Invalid MNIST Image format!"

    buf = file.read(imageWidth * imageHeight * numImages)
    data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
    # The followint normalization is not needed. It is done in CoreML (Preprocessing)    
    # data = (data-127.5)/255.0   # Normalize to [-1..1]
    samples = data.reshape(numImages, imageWidth, imageHeight, 1)

    file = open(labelsFileName, mode='rb')
    header = file.read(8)
    magic, numLabels = struct.unpack(">ii", header)
    assert magic == 2049, "Error: Invalid MNIST Label format!"

    buf = file.read(numLabels)
    labels = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
    return samples, labels

# Update the file names to point to the location of MNIST dataset
testSamples, testLabels = getDataset('/Users/shahab/data/mnist/t10k-images.idx3-ubyte',
                                     '/Users/shahab/data/mnist/t10k-labels.idx1-ubyte')

testSamples = testSamples.reshape(-1,1,28,28)  # CoreML uses Channel-First format

# Inference using the CoreML model
import PIL
import coremltools

coreMlModel = coremltools.models.MLModel(cmlFile)

predictions = []
for sample in testSamples:
    img = PIL.Image.fromarray(np.uint8(sample.reshape(28,28)),'L')
    results = coreMlModel.predict({'InputImage': img})

    predictions += [ int(results["PredictedLabel"]) ]
    
accuracy = float(np.sum(testLabels == predictions))/len(testLabels)
print( "Test Accuracy: %f"%(accuracy) )

Test Accuracy: 0.989100


## Where do I go from here?

[Exporting LeNet-5 Model to ONNX](LeNet5-MNIST-ONNX.ipynb)

[Exporting LeNet-5 Model to TensorFlow](LeNet5-MNIST-TF.ipynb)

---

[Fireball Playgrounds](../Contents.ipynb)

[Handwritten Digit Recognition (LeNet-5/MNIST)](LeNet5-MNIST.ipynb)

[Reducing number of parameters of LeNet-5 Model](LeNet5-MNIST-Reduce.ipynb)

[Pruning LeNet-5 Model](LeNet5-MNIST-Prune.ipynb)

[Quantizing LeNet-5 Model](LeNet5-MNIST-Quantize.ipynb)